# Welcome to the Anemoi Framework in AzureML!

This notebook will guide you through the basic steps of running the Anemoi framework.

1) Create your training and validation datasets
2) Submit a training job
3) Run inference
4) Run verification

More details will be provided in each individual section.

## Step 0: Configure azure settings and permissions

In [ ]:
from azureml.core import Workspace, Experiment
from azure.ai.ml import MLClient, Input, Output, command
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment, BuildContext
from azure.ai.ml.constants import AssetTypes

ws = Workspace.from_config()

ml_client = MLClient(
    DefaultAzureCredential(),
    ws.subscription_id,
    ws.resource_group,
    ws.name,
)

# Get your workspace's default datastore (or specify another registered datastore)
default_ds = ml_client.datastores.get_default()

## Step 1: Create conda environment
You only need to do this once! Check if this environment was already created by someone else before running.

In [ ]:
# create anemoi conda environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="anemoi-conda.yaml",
    name="anemoi",
    description="anemoi environment created from a Docker image plus Conda environment.",
)

ml_client.environments.create_or_update(env_docker_conda)

## Step 2: Create training and validation datasets with ufs2arco

In [ ]:
"""More documentation on submitting jobs this way can be found here:
https://docs.azure.cn/en-us/machine-learning/how-to-train-model?view=azureml-api-2&tabs=python#example-job
This website is where I got almost all code ideas in this notebook!
"""
######### things to edit yourself! ############
experiment_name = 'ufs2arco_testing'
output_path = "replay"
output_zarr = "training.zarr"

# dont edit anything outside of this :)
###############################################

outputs = Output(
    type="uri_folder",
    path=f"azureml://datastores/{default_ds.name}/paths/{output_path}/{output_zarr}" 
)

# This creates the command that will be submitted within the job
command_job = command(
    code=".",
    command=f"bash submit_ufs2arco.sh ${{outputs.output_blob}} {output_path} {output_zarr}",
    environment="anemoi:6",
    compute="Standard-D13-v2",
    experiment_name=experiment_name,
    display_name="test",
    outputs={"output_blob": outputs},
)

# Submit the job
returned_job = ml_client.jobs.create_or_update(command_job)

# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

## Step 3: Submit a training job

In [1]:
# coming soon :)